Dataset link

https://chanzuckerberg.github.io/cellxgene-census/notebooks/api_demo/census_datasets.html

In [1]:
!python --version

Python 3.10.18


In [ ]:
# !pip install cellxgene_census google-generativeai 

In [4]:
import cellxgene_census

census = cellxgene_census.open_soma(census_version="2025-01-30")
census_datasets = census["census_info"]["datasets"].read().concat().to_pandas()

# for convenience, indexing on the soma_joinid which links this to other census data.
census_datasets = census_datasets.set_index("soma_joinid")

census_datasets

,citation,collection_id,collection_name,collection_doi,collection_doi_label,dataset_id,dataset_version_id,dataset_title,dataset_h5ad_path,dataset_total_cell_count
soma_joinid,,,,,,,,,,
0,Publication: https://doi.org/10.1016/j.isci.20...,8e880741-bf9a-4c8e-9227-934204631d2a,High Resolution Slide-seqV2 Spatial Transcript...,10.1016/j.isci.2022.104097,Marshall et al. (2022) iScience,4eb29386-de81-452f-b3c0-e00844e8c7fd,f76861bb-becb-4eb7-82fc-782dc96ccc7f,Spatial transcriptomics in mouse: Puck_191112_05,4eb29386-de81-452f-b3c0-e00844e8c7fd.h5ad,10888
1,Publication: https://doi.org/10.1016/j.isci.20...,8e880741-bf9a-4c8e-9227-934204631d2a,High Resolution Slide-seqV2 Spatial Transcript...,10.1016/j.isci.2022.104097,Marshall et al. (2022) iScience,78d59e4a-82eb-4a61-a1dc-da974d7ea54b,7d7ec1b6-6e3f-4aaa-9442-4b22f3424396,Spatial transcriptomics in mouse: Puck_191112_08,78d59e4a-82eb-4a61-a1dc-da974d7ea54b.h5ad,10250
2,Publication: https://doi.org/10.1016/j.isci.20...,8e880741-bf9a-4c8e-9227-934204631d2a,High Resolution Slide-seqV2 Spatial Transcript...,10.1016/j.isci.2022.104097,Marshall et al. (2022) iScience,add5eb84-5fc9-4f01-982e-a346dd42ee82,de54aed8-4f73-48f6-9229-418a840e2d82,Spatial transcriptomics in mouse: Puck_191109_20,add5eb84-5fc9-4f01-982e-a346dd42ee82.h5ad,12906
3,Publication: https://doi.org/10.1016/j.isci.20...,8e880741-bf9a-4c8e-9227-934204631d2a,High Resolution Slide-seqV2 Spatial Transcript...,10.1016/j.isci.2022.104097,Marshall et al. (2022) iScience,b020294c-ab82-4547-b5a7-63d8ffa575ed,abe4fce1-0859-4a56-ad1e-734d79f0e6c8,Spatial transcriptomics in mouse: Puck_191112_13,b020294c-ab82-4547-b5a7-63d8ffa575ed.h5ad,15161
4,Publication: https://doi.org/10.1038/s41591-02...,a96133de-e951-4e2d-ace6-59db8b3bfb1d,HTAN/HTAPP Broad - Spatio-molecular dissection...,10.1038/s41591-024-03215-z,Klughammer et al. (2024) Nat Med,d7476ae2-e320-4703-8304-da5c42627e71,863fc5e4-bd4a-4681-9c3d-0ee7ef54e327,HTAPP-330-SMP-1082 scRNA-seq,d7476ae2-e320-4703-8304-da5c42627e71.h5ad,565
...,...,...,...,...,...,...,...,...,...,...
1568,Publication: https://doi.org/10.1038/s41586-02...,45d5d2c3-bc28-4814-aed6-0bb6f0e11c82,A single-cell transcriptional timelapse of mou...,10.1038/s41586-024-07069-w,Qiu et al. (2024) Nature,dcfa2614-7ca7-4d82-814c-350626eccb26,3002a659-a1a9-4406-9976-99e658e1fbb5,Major cell cluster: Mesoderm,dcfa2614-7ca7-4d82-814c-350626eccb26.h5ad,3267338
1569,Publication: https://doi.org/10.1126/science.a...,e5f58829-1a66-40b5-a624-9046778e74f5,Tabula Sapiens,10.1126/science.abl4896,The Tabula Sapiens Consortium* et al. (2022) S...,53d208b0-2cfd-4366-9866-c3c6114081bc,10df7690-6d10-4029-a47e-0f071bb2df83,Tabula Sapiens - All Cells,53d208b0-2cfd-4366-9866-c3c6114081bc.h5ad,1136218
1570,Publication: https://doi.org/10.1038/s41586-02...,45d5d2c3-bc28-4814-aed6-0bb6f0e11c82,A single-cell transcriptional timelapse of mou...,10.1038/s41586-024-07069-w,Qiu et al. (2024) Nature,dcfd4feb-18a3-4b30-81d7-1b0c544a8ab3,3817734b-0f82-433b-8c38-55b214200fff,Whole dataset: Raw counts only,dcfd4feb-18a3-4b30-81d7-1b0c544a8ab3.h5ad,11441407


In [5]:
def format_technology_prompt(row):
    prompt = f"""
Based on the following information, identify the **specific RNA-seq technology** used in the experiment.
Your output must be **technology name only**, without any explanation, uncertainty, or additional text.
Do not include use company names, product names, or any other identifiers. (like "10x Genomics", "10x Visium", or "Illumina").

Strictly format your response as:
<exact technology name — no extra words, no sentences, no parentheses>

Dataset Title: {row['dataset_title']}
Citation: {row['citation']}
Collection Name: {row['collection_name']}
""".strip()
    return prompt


In [6]:
# # generate by openrouter API

# import requests, os, json

# def openrouter_chat(model: str, prompt: str):
#     resp = requests.post(
#         "https://openrouter.ai/api/v1/chat/completions",
#         headers={
#             "Authorization": f"Bearer {os.getenv('OPENROUTER_API_KEY')}",
#             "Content-Type": "application/json"
#         },
#         data=json.dumps({
#             "model": model,
#             "messages": [{"role": "user", "content": prompt}]
#         })
#     )
#     return resp.json()

# def generate_openrouter_description(row, model="deepseek/deepseek-r1-0528-qwen3-8b:free"):
#     prompt = format_technology_prompt(row)
#     result = openrouter_chat(model, prompt)
#     try:
#         return result["choices"][0]["message"]["content"].strip()
#     except (KeyError, IndexError) as e:
#         return f"[Error retrieving description: {e}]"
    
# # OpenRouter API to summarize a dataset description

# model = "deepseek/deepseek-r1-0528:free"
# query = format_technology_prompt(census_datasets.iloc[4])
# print("User: \n", query)

# result = openrouter_chat(model, query)
# try:
#     message = result["choices"][0]["message"]["content"]
#     print(f"\nAssistant ({model}): \n {message}")
# except KeyError:
#     print("Error in response format:", result)
#     # raise ValueError("Unexpected response format from OpenRouter API")

In [8]:
model = "models/gemini-1.5-flash-latest"

import os
import google.generativeai as genai
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
genai_model = genai.GenerativeModel(model)

# Gemini API to summarize a dataset description
query = format_technology_prompt(census_datasets.iloc[4])
print("User: \n", query)

result = genai_model.generate_content(query)
try:
    print(f"\nAssistant ({model}): \n {result.text}")
except Exception:
    print("Error in response format:", result)
    # raise ValueError("Unexpected response format from OpenRouter API")

User: 
 Based on the following information, identify the **specific RNA-seq technology** used in the experiment.
Your output must be **technology name only**, without any explanation, uncertainty, or additional text.
Do not include use company names, product names, or any other identifiers. (like "10x Genomics", "10x Visium", or "Illumina").

Strictly format your response as:
<exact technology name — no extra words, no sentences, no parentheses>

Dataset Title: HTAPP-330-SMP-1082 scRNA-seq
Citation: Publication: https://doi.org/10.1038/s41591-024-03215-z Dataset Version: https://datasets.cellxgene.cziscience.com/863fc5e4-bd4a-4681-9c3d-0ee7ef54e327.h5ad curated and distributed by CZ CELLxGENE Discover in Collection: https://cellxgene.cziscience.com/collections/a96133de-e951-4e2d-ace6-59db8b3bfb1d
Collection Name: HTAN/HTAPP Broad - Spatio-molecular dissection of the breast cancer metastatic microenvironment

Assistant (models/gemini-1.5-flash-latest): 
 scRNA-seq



In [43]:
# import pandas as pd

# # The census_datasets DataFrame is large, this is a subset for demonstration
# df = census_datasets[0:10]

# def generate_technology(row):
#     prompt = format_technology_prompt(row)
#     response = genai_model.generate_content(prompt)
#     try:
#         return response.text.strip()
#     except Exception as e:
#         print(f"Error on row: {row['dataset_title']} — {e}")
#         return "ERROR"

# df['technology'] = df.apply(generate_technology, axis=1)

# new_df = pd.DataFrame({
#     'dataset_title': df['dataset_title'],
#     'collection_doi': df['collection_doi'],
#     'dataset_total_cell_count': df['dataset_total_cell_count'],
#     'technology': df['technology']
# })

# new_df.to_csv("census_technology.csv", index=False)
# new_df

In [ ]:
import pandas as pd

model = "models/gemini-1.5-flash-latest"
SAVE_FILE = "census_technology.csv"
UPDATED_FILE = "census_technology_updated.csv"
df = census_datasets

if os.path.exists(UPDATED_FILE):
    existing_df = pd.read_csv(UPDATED_FILE)
    existing_df.to_csv(SAVE_FILE, index=False)
    print(f"Loaded {len(existing_df)} rows from existing file.")
else:
    existing_df = pd.DataFrame(columns=[
        'dataset_title', 'collection_name', 'technology'
    ])
    print("Starting fresh.")


import google.generativeai as genai
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
genai_model = genai.GenerativeModel(model)

start_idx, end_idx = None, None

for idx in range(len(df)):
    if (idx < len(existing_df)) and pd.notna(existing_df.loc[idx, "technology"]) and existing_df.loc[idx, "technology"].strip() not in ["", "ERROR"]:
        continue

    try:
        prompt = format_technology_prompt(df.loc[idx])
        response = genai_model.generate_content(prompt)
        tech = response.text.strip()

        existing_df.loc[idx] = {
            'dataset_title': df.loc[idx, 'dataset_title'],
            'collection_name': df.loc[idx, 'collection_name'],
            'technology': tech
        }

        if start_idx is None:
            start_idx = idx
        end_idx = idx

        print(f"Row {idx} generated: {tech}")

    except Exception as e:
        print(f"Stopped at row {idx} due to error: {e}")
        break

existing_df.to_csv(UPDATED_FILE, index=False)
print(f"\nSaved to: {UPDATED_FILE}")

if start_idx is not None:
    print(f"Generated from index {start_idx} to {end_idx} this run.")
else:
    print("No new rows were generated.")

Loaded 14 rows from existing file.
Row 14 generated: Slide-seqV2
Row 15 generated: Slide-seqV2
Row 16 generated: Slide-seqV2
Row 17 generated: Slide-seqV2
Row 18 generated: Spatial Transcriptomics
Row 19 generated: Spatial transcriptomics
Row 20 generated: Slide-seqV2
Row 21 generated: slide-seqV2
Row 22 generated: scRNA-seq
Row 23 generated: Slide-seqV2
Row 24 generated: Single-cell RNA sequencing
Row 25 generated: Slide-seqV2
Row 26 generated: Spatial transcriptomics
Row 27 generated: Slide-seqV2
Row 28 generated: Spatial transcriptomics
Stopped at row 29 due to error: 429 Resource has been exhausted (e.g. check quota).

Saved to: census_technology_updated.csv
Generated from index 14 to 28 this run.


In [14]:
existing_df

,dataset_title,collection_name,technology
0,Spatial transcriptomics in mouse: Puck_191112_05,High Resolution Slide-seqV2 Spatial Transcript...,Slide-seqV2
1,Spatial transcriptomics in mouse: Puck_191112_08,High Resolution Slide-seqV2 Spatial Transcript...,Slide-seqV2
2,Spatial transcriptomics in mouse: Puck_191109_20,High Resolution Slide-seqV2 Spatial Transcript...,Slide-seqV2
3,Spatial transcriptomics in mouse: Puck_191112_13,High Resolution Slide-seqV2 Spatial Transcript...,Slide-seqV2
4,HTAPP-330-SMP-1082 scRNA-seq,HTAN/HTAPP Broad - Spatio-molecular dissection...,scRNA-seq
5,Spatial transcriptomics in mouse: Puck_191109_14,High Resolution Slide-seqV2 Spatial Transcript...,Slide-seqV2
6,Healthy human liver: B cells,"Single-Cell, Single-Nucleus, and Spatial RNA S...",Single-cell RNA sequencing
7,thymus scRNA-seq atlas - myeloid p2 subset,A spatial human thymus cell atlas mapped to a ...,scRNA-seq
8,Spatial transcriptomics in mouse: Puck_191109_18,High Resolution Slide-seqV2 Spatial Transcript...,Slide-seqV2
9,Human tonsil nonlymphoid cells scRNA,Single-cell analysis of human B cell maturatio...,scRNA-seq
